<a href="https://colab.research.google.com/github/whosrara/tpsc/blob/main/yolov5_coba_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Playground (work)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cv2
import torch
from collections import defaultdict

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5m', pretrained=True)

# Dictionary to store information about tracked vehicles
tracked_vehicles = defaultdict(lambda: {'frame_count': 0, 'time_visible': 0, 'bbox': None})

# Function to calculate Intersection over Union (IoU)
def calculate_iou(box1, box2):
    xA = max(box1[0], box2[0])
    yA = max(box1[1], box2[1])
    xB = min(box1[2], box2[2])
    yB = min(box1[3], box2[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)
    box1Area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2Area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    iou = interArea / float(box1Area + box2Area - interArea)

    return iou

# Function to count vehicles in a frame and update tracking information
def count_and_track_vehicles(frame, fps):
    results = model(frame)
    vehicle_classes = ['car', 'motorcycle']  # Define vehicle classes
    vehicles = [x for x in results.pred[0] if results.names[int(x[5])] in vehicle_classes]

    current_frame_vehicles = {}

    for *box, _, class_id in vehicles:
        label = results.names[int(class_id)]
        box = list(box)

        best_match_id = None
        best_iou = 0
        for vehicle_id, data in tracked_vehicles.items():
            iou = calculate_iou(data['bbox'], box)
            if iou > best_iou:
                best_iou = iou
                best_match_id = vehicle_id

        if best_match_id and best_iou > 0.5:  # IoU threshold
            tracked_vehicles[best_match_id]['frame_count'] += 1
            tracked_vehicles[best_match_id]['bbox'] = box
            vehicle_id = best_match_id
        else:
            vehicle_id = len(tracked_vehicles) + 1
            tracked_vehicles[vehicle_id] = {'frame_count': 1, 'bbox': box, 'label': label}

        tracked_vehicles[vehicle_id]['time_visible'] = tracked_vehicles[vehicle_id]['frame_count'] / fps
        current_frame_vehicles[vehicle_id] = tracked_vehicles[vehicle_id]

        cv2.rectangle(frame, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0, 255, 0), 2)
        cv2.putText(frame, f"{label} - {tracked_vehicles[vehicle_id]['time_visible']:.2f} sec", (int(box[0]), int(box[1]-10)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    return len(current_frame_vehicles), current_frame_vehicles, results

# Process video
# video = cv2.VideoCapture('v.mp4')
video = cv2.VideoCapture('j.mp4')
fps = video.get(cv2.CAP_PROP_FPS)  # Get the frame rate of the video

while video.isOpened():
    ret, frame = video.read()
    if not ret:
        break

    vehicle_count, current_frame_vehicles, results = count_and_track_vehicles(frame, fps)

    cv2.putText(frame, f'Tracked Vehicles: {vehicle_count}', (10, 650), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    # for vehicle_id, data in current_frame_vehicles.items():
    #     time_visible = data['time_visible']
    #     cv2.putText(frame, f'ID {vehicle_id}: {time_visible:.2f} sec', (10, 60 + 30 * vehicle_id), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

    cv2.imshow('Frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()

Using cache found in C:\Users\User/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-8-17 Python-3.11.5 torch-2.1.0+cpu CPU

Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients, 48.9 GFLOPs
Adding AutoShape... 


# Playground 2

In [ ]:
import cv2
import torch
from collections import defaultdict

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5m', pretrained=True)

# Dictionary to store information about tracked vehicles with grace period feature
grace_period = 10  # Number of frames a vehicle can be untracked before being removed
tracked_vehicles = defaultdict(lambda: {'frame_count': 0, 'time_visible': 0, 'bbox': None, 'grace_counter': grace_period})

# Function to calculate Intersection over Union (IoU)
def calculate_iou(box1, box2):
    xA = max(box1[0], box2[0])
    yA = max(box1[1], box2[1])
    xB = min(box1[2], box2[2])
    yB = min(box1[3], box2[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)
    box1Area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2Area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    iou = interArea / float(box1Area + box2Area - interArea)

    return iou

# Function to count vehicles in a frame and update tracking information with grace period logic
def count_and_track_vehicles(frame, fps):
    results = model(frame)
    vehicle_classes = ['car', 'motorcycle']  # Define vehicle classes
    vehicles = [x for x in results.pred[0] if results.names[int(x[5])] in vehicle_classes]

    # Decrease grace period counters for all tracked vehicles
    for vehicle_id in list(tracked_vehicles.keys()):
        tracked_vehicles[vehicle_id]['grace_counter'] -= 1
        if tracked_vehicles[vehicle_id]['grace_counter'] <= 0:
            del tracked_vehicles[vehicle_id]

    current_frame_vehicles = {}

    for *box, _, class_id in vehicles:
        label = results.names[int(class_id)]
        box = list(map(int, box))

        best_match_id = None
        best_iou = 0
        for vehicle_id, data in tracked_vehicles.items():
            iou = calculate_iou(data['bbox'], box)
            if iou > best_iou:
                best_iou = iou
                best_match_id = vehicle_id

        if best_match_id and best_iou > 0.5:  # IoU threshold
            tracked_vehicles[best_match_id]['frame_count'] += 1
            tracked_vehicles[best_match_id]['bbox'] = box
            tracked_vehicles[best_match_id]['grace_counter'] = grace_period
            vehicle_id = best_match_id
        else:
            vehicle_id = len(tracked_vehicles) + 1
            tracked_vehicles[vehicle_id] = {'frame_count': 1, 'bbox': box, 'label': label, 'grace_counter': grace_period}

        # Adjusting the frame count for skipped frames
        adjusted_frame_count = tracked_vehicles[vehicle_id]['frame_count'] * frame_skip

        # Calculating time visible with adjusted frame count
        tracked_vehicles[vehicle_id]['time_visible'] = adjusted_frame_count / fps

        current_frame_vehicles[vehicle_id] = tracked_vehicles[vehicle_id]

        cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
        cv2.putText(frame, f"{label} - {tracked_vehicles[vehicle_id]['time_visible']:.2f} sec", (box[0], box[1] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    return len(current_frame_vehicles), current_frame_vehicles, results

# Process video
video = cv2.VideoCapture('v.mp4')
fps = video.get(cv2.CAP_PROP_FPS)  # Get the frame rate of the video

frame_skip = 2  # Skip every 2 frames
frame_count = 0

while video.isOpened():
    ret, frame = video.read()
    if not ret:
        break

    frame_count += 1
    if frame_count % frame_skip != 0:
        continue  # Skip this frame

    vehicle_count, current_frame_vehicles, results = count_and_track_vehicles(frame, fps)

    cv2.putText(frame, f'Tracked Vehicles: {vehicle_count}', (10, 650), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    cv2.imshow('Frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()

Using cache found in C:\Users\User/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-8-17 Python-3.11.5 torch-2.1.0+cpu CPU

Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients, 48.9 GFLOPs
Adding AutoShape... 


# DUMP

In [ ]:
import cv2
import torch
from collections import defaultdict

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Dictionary to store information about tracked vehicles
tracked_vehicles = defaultdict(lambda: {'frame_count': 0, 'time_visible': 0, 'bbox': None})

# Function to calculate Intersection over Union (IoU)
def calculate_iou(box1, box2):
    xA = max(box1[0], box2[0])
    yA = max(box1[1], box2[1])
    xB = min(box1[2], box2[2])
    yB = min(box1[3], box2[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)
    box1Area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2Area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    iou = interArea / float(box1Area + box2Area - interArea)

    return iou

# Function to count vehicles in a frame and update tracking information
def count_and_track_vehicles(frame, fps):
    results = model(frame)
    vehicle_classes = ['car', 'motorcycle']  # Define vehicle classes
    vehicles = [x for x in results.pred[0] if results.names[int(x[5])] in vehicle_classes]

    current_frame_vehicles = {}

    for *box, _, class_id in vehicles:
        label = results.names[int(class_id)]
        box = list(map(int, box))

        best_match_id = None
        best_iou = 0
        for vehicle_id, data in tracked_vehicles.items():
            iou = calculate_iou(data['bbox'], box)
            if iou > best_iou:
                best_iou = iou
                best_match_id = vehicle_id

        if best_match_id and best_iou > 0.5:  # IoU threshold
            tracked_vehicles[best_match_id]['frame_count'] += 1
            tracked_vehicles[best_match_id]['bbox'] = box
            vehicle_id = best_match_id
        else:
            vehicle_id = len(tracked_vehicles) + 1
            tracked_vehicles[vehicle_id] = {'frame_count': 1, 'bbox': box, 'label': label}

        tracked_vehicles[vehicle_id]['time_visible'] = tracked_vehicles[vehicle_id]['frame_count'] / fps

        # Check if the vehicle has been visible for more than 60 seconds
        if tracked_vehicles[vehicle_id]['time_visible'] > 60:
            bbox_color = (0, 0, 255)  # Red color for more than 1 minute
        else:
            bbox_color = (0, 255, 0)  # Green color for less than 1 minute

        cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), bbox_color, 2)
        cv2.putText(frame, f"{label} - {tracked_vehicles[vehicle_id]['time_visible']:.2f} sec",
                    (box[0], box[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, bbox_color, 2)

        current_frame_vehicles[vehicle_id] = tracked_vehicles[vehicle_id]

    return len(current_frame_vehicles), current_frame_vehicles, results

# Process video
# video = cv2.VideoCapture('j.mp4')
video = cv2.VideoCapture('v.mp4')
fps = video.get(cv2.CAP_PROP_FPS)  # Get the frame rate of the video

while video.isOpened():
    ret, frame = video.read()
    if not ret:
        break

    vehicle_count, current_frame_vehicles, results = count_and_track_vehicles(frame, fps)

    # Check for starvation and count vehicles visible for over 60 seconds
    starvation = False
    starvation_count = 0
    for vehicle_id, data in current_frame_vehicles.items():
        if data['time_visible'] > 60:
            starvation = True
            starvation_count += 1

    # Display tracked vehicles count
    cv2.putText(frame, f'Tracked Vehicles: {vehicle_count}', (10, 650), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Display starvation information on the bottom right
    cv2.putText(frame, f"Starvation: {starvation} (Count: {starvation_count})", (800, 650), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    cv2.imshow('Frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()

Using cache found in C:\Users\User/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-8-17 Python-3.11.5 torch-2.1.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


# Saving

In [ ]:
import cv2
import torch
from collections import defaultdict

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5m', pretrained=True)

# Dictionary to store information about tracked vehicles
tracked_vehicles = defaultdict(lambda: {'frame_count': 0, 'time_visible': 0, 'bbox': None})

# Function to calculate Intersection over Union (IoU)
def calculate_iou(box1, box2):
    xA = max(box1[0], box2[0])
    yA = max(box1[1], box2[1])
    xB = min(box1[2], box2[2])
    yB = min(box1[3], box2[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)
    box1Area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2Area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    iou = interArea / float(box1Area + box2Area - interArea)

    return iou

# Function to count vehicles in a frame and update tracking information
def count_and_track_vehicles(frame, fps):
    results = model(frame)
    vehicle_classes = ['car', 'motorcycle']  # Define vehicle classes
    vehicles = [x for x in results.pred[0] if results.names[int(x[5])] in vehicle_classes]

    current_frame_vehicles = {}

    for *box, _, class_id in vehicles:
        label = results.names[int(class_id)]
        box = list(map(int, box))

        best_match_id = None
        best_iou = 0
        for vehicle_id, data in tracked_vehicles.items():
            iou = calculate_iou(data['bbox'], box)
            if iou > best_iou:
                best_iou = iou
                best_match_id = vehicle_id

        if best_match_id and best_iou > 0.5:  # IoU threshold
            tracked_vehicles[best_match_id]['frame_count'] += 1
            tracked_vehicles[best_match_id]['bbox'] = box
            vehicle_id = best_match_id
        else:
            vehicle_id = len(tracked_vehicles) + 1
            tracked_vehicles[vehicle_id] = {'frame_count': 1, 'bbox': box, 'label': label}

        tracked_vehicles[vehicle_id]['time_visible'] = tracked_vehicles[vehicle_id]['frame_count'] / fps

        # Check if the vehicle has been visible for more than 60 seconds
        if tracked_vehicles[vehicle_id]['time_visible'] > 60:
            bbox_color = (0, 0, 255)  # Red color for more than 1 minute
        else:
            bbox_color = (0, 255, 0)  # Green color for less than 1 minute

        cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), bbox_color, 2)
        cv2.putText(frame, f"{label} - {tracked_vehicles[vehicle_id]['time_visible']:.2f} sec",
                    (box[0], box[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, bbox_color, 2)

        current_frame_vehicles[vehicle_id] = tracked_vehicles[vehicle_id]

    return len(current_frame_vehicles), current_frame_vehicles, results

# Process video
video = cv2.VideoCapture('j.mp4')
# video = cv2.VideoCapture('v.mp4')
fps = video.get(cv2.CAP_PROP_FPS)  # Get the frame rate of the video

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, fps, (int(video.get(3)), int(video.get(4))))

while video.isOpened():
    ret, frame = video.read()
    if not ret:
        break

    vehicle_count, current_frame_vehicles, results = count_and_track_vehicles(frame, fps)

    # Check for starvation and count vehicles visible for over 60 seconds
    starvation = False
    starvation_count = 0
    for vehicle_id, data in current_frame_vehicles.items():
        if data['time_visible'] > 60:
            starvation = True
            starvation_count += 1

    # Display tracked vehicles count
    cv2.putText(frame, f'Tracked Vehicles: {vehicle_count}', (10, 650), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Display starvation information on the bottom right
    cv2.putText(frame, f"Starvation: {starvation} (Count: {starvation_count})", (800, 650), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Write the frame with annotations to the output video
    out.write(frame)

    cv2.imshow('Frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the VideoWriter and VideoCapture objects
out.release()
video.release()
cv2.destroyAllWindows()

Using cache found in C:\Users\User/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-8-17 Python-3.11.5 torch-2.1.0+cpu CPU

Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients, 48.9 GFLOPs
Adding AutoShape... 


In [ ]:
import cv2
import torch
from collections import defaultdict
import numpy as np

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5m', pretrained=True)

# Dictionary to store information about tracked vehicles
tracked_vehicles = defaultdict(lambda: {'frame_count': 0, 'time_visible': 0, 'bbox': None})

# Function to calculate Intersection over Union (IoU)
def calculate_iou(box1, box2):
    xA = max(box1[0], box2[0])
    yA = max(box1[1], box2[1])
    xB = min(box1[2], box2[2])
    yB = min(box1[3], box2[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)
    box1Area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2Area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    iou = interArea / float(box1Area + box2Area - interArea)

    return iou

# Definisikan koordinat untuk area yang ingin diproses
process_area = np.array([(316, 630), (708, 643), (662, 176), (489, 174)])

# Fungsi untuk memeriksa apakah titik berada di dalam area yang ditentukan
def is_point_in_area(point, area):
    return cv2.pointPolygonTest(area, (point[0], point[1]), False) >= 0

# Function to count vehicles in a frame and update tracking information
def count_and_track_vehicles(frame, fps):
    results = model(frame)
    vehicle_classes = ['car', 'motorcycle']  # Define vehicle classes
    vehicles = [x for x in results.pred[0] if results.names[int(x[5])] in vehicle_classes]

    current_frame_vehicles = {}

    for *box, _, class_id in vehicles:
        label = results.names[int(class_id)]
        box = list(map(int, box))

        center_point = [(box[0] + box[2]) // 2, (box[1] + box[3]) // 2]
        if not is_point_in_area(center_point, process_area):
            continue  # Skip if not within specified area

        best_match_id = None
        best_iou = 0
        for vehicle_id, data in tracked_vehicles.items():
            iou = calculate_iou(data['bbox'], box)
            if iou > best_iou:
                best_iou = iou
                best_match_id = vehicle_id

        if best_match_id and best_iou > 0.5:  # IoU threshold
            tracked_vehicles[best_match_id]['frame_count'] += 1
            tracked_vehicles[best_match_id]['bbox'] = box
            vehicle_id = best_match_id
        else:
            vehicle_id = len(tracked_vehicles) + 1
            tracked_vehicles[vehicle_id] = {'frame_count': 1, 'bbox': box, 'label': label}

        tracked_vehicles[vehicle_id]['time_visible'] = tracked_vehicles[vehicle_id]['frame_count'] / fps

        if tracked_vehicles[vehicle_id]['time_visible'] > 60:
            bbox_color = (0, 0, 255)  # Red for more than 1 minute
        else:
            bbox_color = (0, 255, 0)  # Green for less than 1 minute

        cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), bbox_color, 2)
        cv2.putText(frame, f"{label} - {tracked_vehicles[vehicle_id]['time_visible']:.2f} sec",
                    (box[0], box[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, bbox_color, 2)

        current_frame_vehicles[vehicle_id] = tracked_vehicles[vehicle_id]

    return len(current_frame_vehicles), current_frame_vehicles, results

# Process video
video = cv2.VideoCapture('j.mp4')
fps = video.get(cv2.CAP_PROP_FPS)  # Get the frame rate of the video

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, fps, (int(video.get(3)), int(video.get(4))))

while video.isOpened():
    ret, frame = video.read()
    if not ret:
        break

    vehicle_count, current_frame_vehicles, results = count_and_track_vehicles(frame, fps)

    # Check for starvation and count vehicles visible for over 60 seconds
    starvation = False
    starvation_count = 0
    for vehicle_id, data in current_frame_vehicles.items():
        if data['time_visible'] > 60:
            starvation = True
            starvation_count += 1

    # Display tracked vehicles count
    cv2.putText(frame, f'Tracked Vehicles: {vehicle_count}', (10, 650), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Display starvation information on the bottom right
    cv2.putText(frame, f"Starvation: {starvation} (Count: {starvation_count})", (800, 650), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Draw process area
    cv2.polylines(frame, [process_area], True, (255, 255, 255), 3)  # White lines

    # Write the frame with annotations to the output video
    out.write(frame)

    cv2.imshow('Frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the VideoWriter and VideoCapture objects
out.release()
video.release()
cv2.destroyAllWindows()

/usr/local/lib/python3.10/dist-packages/torch/hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


requirements: Ultralytics requirements ['gitpython>=3.1.30', 'Pillow>=10.0.1'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 146.1 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0

requirements: AutoUpdate success ✅ 8.9s, installed 2 packages: ['gitpython>=3.1.30', 'Pillow>=10.0.1']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



YOLOv5 🚀 2023-12-14 Python-3.10.12 torch-2.1.0+cu118 CPU

100%|██████████| 40.8M/40.8M [00:00<00:00, 190MB/s]

Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients, 48.9 GFLOPs
Adding AutoShape... 


# POINT

In [ ]:
import cv2

# Fungsi callback untuk menangani klik mouse
def click_event(event, x, y, flags, params):
    if event == cv2.EVENT_LBUTTONDOWN:
        # Menyimpan koordinat ketika mouse diklik
        points.append((x, y))

        # Menampilkan titik yang telah diklik pada frame
        cv2.circle(frame, (x, y), 5, (0, 0, 255), -1)
        cv2.imshow("Frame", frame)

        # Jika sudah ada 4 titik, tampilkan koordinatnya
        if len(points) == 4:
            print("Koordinat titik yang dipilih:", points)

# Load video
video = cv2.VideoCapture('j.mp4')

# Baca frame pertama dari video
ret, frame = video.read()

if not ret:
    print("Gagal membaca video")
    video.release()
else:
    # Buat jendela untuk menampilkan frame
    cv2.namedWindow("Frame")
    cv2.setMouseCallback("Frame", click_event)

    points = []  # Untuk menyimpan koordinat titik yang dipilih
    cv2.imshow("Frame", frame)
    cv2.waitKey(0)

    # Tutup semua jendela dan release video setelah selesai
    cv2.destroyAllWindows()
    video.release()


Koordinat titik yang dipilih: [(316, 630), (708, 643), (662, 176), (489, 174)]
